In [19]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re


def _get_soup(url: str):
    # URLからHTMLを取得
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    get_soup = BeautifulSoup(response.text, "html.parser")

    return get_soup


def clean_text(text):
    if not text:
        return ""
    # 改行と余分な空白を整理
    text = re.sub(r"\s+", " ", text.strip())
    return text


def scrape_movie_info(url):
    soup = _get_soup(url)
    movies_data = []

    # 全ての.box要素を取得
    movie_boxes = soup.find_all("div", class_="box")

    for box in movie_boxes:
        try:
            # 通常の映画タイトルと特集タイトルの両方を確認
            title_elem = box.find("span", class_="eiga-title")
            title_s_elem = box.find("span", class_="title-s")

            # タイトルが見つからない場合はスキップ
            if not title_elem and not title_s_elem:
                continue

            title = clean_text(title_elem.text if title_elem else title_s_elem.text)

            # 特集作品の場合は特別な処理
            if title_s_elem:
                # 特集内の作品情報を取得
                details_elem = box.find("details")
                if details_elem:
                    sub_movies = details_elem.find_all("div", class_="box")
                    for sub_movie in sub_movies:
                        sub_title_elem = sub_movie.find("span", class_="eiga-title")
                        if not sub_title_elem:
                            continue

                        movie_info = process_movie_box(
                            sub_movie, is_special=True, special_title=title
                        )
                        if movie_info:
                            movies_data.append(movie_info)
                else:
                    movie_info = process_movie_box(box, is_special=True)
                    if movie_info:
                        movies_data.append(movie_info)
            else:
                # 通常の映画情報を処理
                movie_info = process_movie_box(box)
                if movie_info:
                    movies_data.append(movie_info)

        except Exception as e:
            print(f"Error processing movie: {e}")

    # DataFrameを作成
    df = pd.DataFrame(movies_data)

    # カラムの順序を整理
    columns = [
        "タイトル",
        "特集名",
        "制作年/国/時間",
        "監督/出演",
        "概要",
        "上映スケジュール",
        "料金",
        "受賞歴",
        "イベント情報",
    ]
    df = df.reindex(columns=columns)

    return df


def process_movie_box(box, is_special=False, special_title=None):
    # 基本情報を取得
    title_elem = box.find("span", class_="eiga-title")
    stuff_elem = box.find(["p", "span"], class_="stuff")
    note_elem = box.find(["p", "span"], class_="note")
    day_elem = box.find(["p", "span"], class_="day")
    price_elem = box.find("p", class_="price")
    award_elem = box.find("p", class_="syo")
    event_elem = box.find("p", class_="p3")

    # タイトルがない場合はスキップ
    if not title_elem:
        return None

    # 基本情報を整理
    basic_info = ""
    director_cast = ""
    if stuff_elem:
        stuff_text = stuff_elem.text.strip()
        stuff_lines = stuff_text.split("\n")
        basic_info = clean_text(stuff_lines[0]) if stuff_lines else ""
        director_cast = (
            clean_text("\n".join(stuff_lines[1:])) if len(stuff_lines) > 1 else ""
        )

    return {
        "タイトル": clean_text(title_elem.text),
        "特集名": clean_text(special_title) if is_special else "",
        "制作年/国/時間": basic_info,
        "監督/出演": director_cast,
        "概要": clean_text(note_elem.text) if note_elem else "",
        "上映スケジュール": clean_text(day_elem.text) if day_elem else "",
        "料金": clean_text(price_elem.text) if price_elem else "",
        "受賞歴": clean_text(award_elem.text) if award_elem else "",
        "イベント情報": clean_text(event_elem.text) if event_elem else "",
    }


def process_html_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    return scrape_movie_info(html_content)

In [ ]:
html_content = "https://shimotakaidocinema.com/schedule/schedule.html"

df = scrape_movie_info(html_content)
df.to_csv("../data/shimotakaido_cinema.csv", index=False, encoding="utf-8")
df.head()

,タイトル,特集名,制作年/国/時間,監督/出演,概要,上映スケジュール,料金,受賞歴,イベント情報
0,リュミエール,,1976年/フランス/1h42,監督・脚本：ジャンヌ・モロー 出演：ジャンヌ・モロー、ルチア・ボゼー、フランシーヌ・ラセット...,サラ、ラウラ、ジュリエンヌ、キャロリーヌ。4人の「女優」たちの欲望、葛藤、そして連帯が鮮やか...,2/8(土)、2/12(水)、2/14(金) 9:30〜(終11:15) 2/11(火・祝)...,,,
1,思春期,,1979年/フランス/1h34,監督・脚本：ジャンヌ・モロー 出演：レティシア・ショヴォー、シモーヌ・シニョレ、フランシス・...,戦争の影が迫る1939年。フランス中部の村で12歳のマリーが母、祖母と共に過ごした。マリーは...,2/9(日)、2/13(木) 10:45〜(終12:22) 2/10(月) 9:30〜(終1...,,,
2,リリアン・ギッシュの肖像,,1983年/フランス/0h59,監督・脚本：ジャンヌ・モロー 出演：リリアン・ギッシュ、ジャンヌ・モロー,1983年の夏、ニューヨーク。サイレント映画期から活躍し、ハリウッドの頂点を極めたリリアン・...,2/8(土)、2/12(水)、2/14(金) 11:25〜(終12:27) 2/9(日)、2...,,,
3,ジョイランド わたしの願い,,2022年/パキスタン/2h07,監督・脚本：サーイム・サーディク 出演：アリ・ジュネージョー、ラスティ・ファルーク,パキスタンの大都市ラホール。トランスジェンダー女性のダンサーに恋する夫、働き続けたい妻。若き...,2/8(土)～2/14(金) 12：40～(終14：50),一般1700円 / 大学・専門1300円 / シニア1100円 / 小・中・高1000円 /...,カンヌ国際映画祭「ある視点」部門審査員賞・クィア・パルム賞,
4,ルート29,,2024年/日本/2h00,監督・脚本：森井勇佑『こちらあみ子』 原作：中尾太一「ルート29、解放」 音楽：Bialys...,清掃員・のり子は仕事先の入院患者から娘ハルを連れてきてほしいと頼まれ…。姫路から鳥取まで１本...,2/8(土)～2/14(金) 15：05～(終17：10),一般1700円 / 大学・専門1300円 / シニア1100円 / 小・中・高1000円 /...,,★2/8(土)上映後、森井勇佑監督による舞台挨拶あり


In [18]:
print(df.head(10).to_markdown())

|    | タイトル                                 | 制作年/国/時間                  | 概要                                                                                                                                                       | 詳細情報                                                                               | 上映スケジュール                      | 料金                                                                        | 受賞歴                                                           |
|---:|:-----------------------------------------|:--------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|:--------------------------------------|:----------------------------------------------------------------------------|:-----------------------------------------------------------------|

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def upload_to_sheets(df, credentials_path, spreadsheet_name):
    """
    DataFrameをGoogle Sheetsにアップロード

    Parameters:
    - df: アップロードするDataFrame
    - credentials_path: Google Cloud Platformで取得した認証情報のJSONファイルパス
    - spreadsheet_name: アップロード先のスプレッドシート名
    """
    # APIの認証設定
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        credentials_path, scope
    )
    gc = gspread.authorize(credentials)

    try:
        # スプレッドシートを開く（なければ作成）
        try:
            workbook = gc.open(spreadsheet_name)
        except gspread.SpreadsheetNotFound:
            workbook = gc.create(spreadsheet_name)

        # 新しいワークシートを作成（日付付き）
        sheet_name = f"映画情報_{datetime.now().strftime('%Y%m%d')}"
        try:
            worksheet = workbook.add_worksheet(
                title=sheet_name, rows=len(df) + 1, cols=len(df.columns)
            )
        except gspread.exceptions.APIError:
            # 同名のシートが存在する場合は、既存のものを使用
            worksheet = workbook.worksheet(sheet_name)
            worksheet.clear()  # 既存データをクリア

        # ヘッダーとデータを準備
        headers = df.columns.tolist()
        data = df.values.tolist()

        # ヘッダーを更新
        worksheet.update("A1", [headers])

        # データを更新
        if data:
            worksheet.update("A2", data)

        # 列幅の自動調整
        worksheet.columns_auto_resize(0, len(headers))

        print(
            f"データを {spreadsheet_name} の {sheet_name} シートにアップロードしました。"
        )
        return workbook.url

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None